In [2]:
import os
import time
import markdown2
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from autogen_core.tools import FunctionTool
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from autogen_ext.models.openai import OpenAIChatCompletionClient
import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv

load_dotenv()

model_client = OpenAIChatCompletionClient(
    # model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    api_key=os.getenv("GEMINI_API_KEY"),
    api_type="google",
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
    },
)

def generate_pdf(content: str, filename: str = "output.pdf", output_dir: str = "output") -> str:
    """Generate a PDF file from text content with proper word wrapping and markdown support."""
    
    os.makedirs(output_dir, exist_ok=True)

    # Ensure the filename has a .pdf extension
    if not filename.lower().endswith(".pdf"):
        filename += ".pdf"

    filepath = os.path.join(output_dir, filename)

    # Convert Markdown to HTML
    html_content = markdown2.markdown(content)

    # Create a PDF document
    doc = SimpleDocTemplate(filepath, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Convert HTML to ReportLab Paragraph
    for paragraph in html_content.split("\n"):
        if paragraph.strip():  # Ignore empty lines
            story.append(Paragraph(paragraph, styles["Normal"]))
            story.append(Spacer(1, 12))  # Add spacing between paragraphs

    # Build the PDF
    doc.build(story)

    return filepath

def google_search(query: str, num_results: int = 2, max_chars: int = 500, max_attempts: int = 2) -> list:
    """Search Google for patents, keywords and return results with a stopping condition."""
    
    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    
    attempts = 0
    results = []

    while attempts < max_attempts and len(results) < num_results:
        params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(response.json())
            raise Exception(f"Error in API request: {response.status_code}")

        new_results = response.json().get("items", [])
        results.extend(new_results[: num_results - len(results)])

        if len(results) >= num_results:
            break  # Stop searching if enough results are found

        attempts += 1
        time.sleep(1)  # Be respectful to API rate limits

    return results



def arxiv_search(query: str, max_results: int = 2, max_attempts: int = 2) -> list:
    """Search Arxiv for papers and return results with a stopping condition."""
    import arxiv

    client = arxiv.Client()
    
    attempts = 0
    results = []

    while attempts < max_attempts and len(results) < max_results:
        search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

        for paper in client.results(search):
            if len(results) >= max_results:
                break
            results.append({
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            })

        if len(results) >= max_results:
            break  # Stop searching if enough results are found
        
        attempts += 1
        time.sleep(1)  # Avoid overloading ArXiv API

    return results

google_search_tool = FunctionTool(
    google_search,
    name="google_search",
    description="Search Google for information, returning results with snippets and body content."
)

arxiv_search_tool = FunctionTool(
    arxiv_search,
    name="arxiv_search",
    description="Search Arxiv for publications related to a given topic, including abstracts citation and pdf links."
)

pdf_generator_tool = FunctionTool(
    generate_pdf,
    name="PDF_Generator",
    description="Generate a PDF document from the content generated by Researcher and save it in the output folder with markdown support."
)

research_agent = AssistantAgent(
    name="Researcher",
    model_client=model_client,
    description="Research specialist for finding information",
    tools=[google_search_tool, arxiv_search_tool],
    system_message="You are a research specialist. Use the provided google_search and arxiv_search tools to find information about the publication."
)

report_agent_gemini = AssistantAgent(
    name="ReporterGenerator",
    model_client=model_client,
    description="Generate a technical literature review document for the given topic by collaborating with pdf_generator_tool, google_search_tool, arxiv_search_tool tools",
    tools=[pdf_generator_tool, google_search_tool, arxiv_search_tool],
    system_message=(
        "You are a helpful assistant. Your task is to synthesize data into a high-quality literature review and save output as PDF."
        "Use the google_search_tool, arxiv_search_tool tools to generate the technical report including links and citation."
        "Use the PDF_Generator tool to save the final PDF reports."
        "Your response should end with the word 'TERMINATE'."
    )
)

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

termination_condition = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[report_agent_gemini],
    termination_condition=termination_condition
)

await Console(
    team.run_stream(
        task="Write a literature review on 'Advanced features on Earbuds or AirPods over the last 2 years' and save it as a PDF named anc.pdf",
    )
)

---------- user ----------
Generate a literature review document on the topic, 'advancement on ANC for hearables devices' and save it as a PDF named anc.pdf
---------- ReporterGenerator ----------
[FunctionCall(id='', arguments='{"query":"advancement on ANC for hearables devices","max_results":5}', name='arxiv_search'), FunctionCall(id='', arguments='{"query":"advancement on ANC for hearables devices","num_results":5}', name='google_search')]
---------- ReporterGenerator ----------
[FunctionExecutionResult(content='[{\'title\': \'EarCough: Enabling Continuous Subject Cough Event Detection on Hearables\', \'authors\': [\'Xiyuxing Zhang\', \'Yuntao Wang\', \'Jingru Zhang\', \'Yaqing Yang\', \'Shwetak Patel\', \'Yuanchun Shi\'], \'published\': \'2023-03-18\', \'abstract\': \'Cough monitoring can enable new individual pulmonary health applications.\\nSubject cough event detection is the foundation for continuous cough\\nmonitoring. Recently, the rapid growth in smart hearables has opened n

Error processing publish message for ReporterGenerator/04b12a65-746a-4543-b723-9e479519483c
Traceback (most recent call last):
  File "/Users/hissain/git/github/llm/llm_literature/.venv/lib/python3.11/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 505, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hissain/git/github/llm/llm_literature/.venv/lib/python3.11/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hissain/git/github/llm/llm_literature/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 48, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hissain/git/github/llm/llm_literature/.venv/lib/python3.11/site-packages/autogen_core/_r

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Generate a literature review document on the topic, 'advancement on ANC for hearables devices' and save it as a PDF named anc.pdf", type='TextMessage'), ToolCallRequestEvent(source='ReporterGenerator', models_usage=RequestUsage(prompt_tokens=209, completion_tokens=30), content=[FunctionCall(id='', arguments='{"query":"advancement on ANC for hearables devices","max_results":5}', name='arxiv_search'), FunctionCall(id='', arguments='{"query":"advancement on ANC for hearables devices","num_results":5}', name='google_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='ReporterGenerator', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': \'EarCough: Enabling Continuous Subject Cough Event Detection on Hearables\', \'authors\': [\'Xiyuxing Zhang\', \'Yuntao Wang\', \'Jingru Zhang\', \'Yaqing Yang\', \'Shwetak Patel\', \'Yuanchun Shi\'], \'published\': \'2023-03-18\', \'abstr